# INTRODUCTION

In this notebook we want to assess the optimal weights to ensemble our best models that are already trained. The problem is that we can not simply test several weights ona a test set since our models have been trained on different training sets, so it is possible that some models were trained on data that are part of the test set. To overcome the latter problem we are going to build a more consistent evaluation of the models:

- each model is tested on 7 different datasets, and the performance is simply the avarage performance obtained over the seven datasets

- each dataset is a balanced dataset (same number of time series for each category)

- each dataset comprise 277 element for category (cardinality of category F, less represented category). Therefore all the seven datasets contains all the time series from category F and subsets of 277 elements for other categories.

- the subsets of 277 elements are built by means of random choice with a different seed for every dataset.



# IMPORT PACKAGES AND DATA

In [1]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [2]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import initializers
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.13.0


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [4]:
data=np.load('/kaggle/input/data-assignment2/training_data.npy', allow_pickle=True)

In [5]:
categories=np.load('/kaggle/input/data-assignment2/categories.npy', allow_pickle=True)

In [6]:
valid_periods=np.load('/kaggle/input/data-assignment2/valid_periods.npy', allow_pickle=True)

In [7]:
recovered_series = []

for i in range(len(data)):
    start, end = valid_periods[i]
    series = data[i][start:end]
    recovered_series.append(series)

categorized_series = {category: [] for category in ['A', 'B', 'C', 'D', 'E', 'F']}

for i, category_code in enumerate(categories):
    category = category_code.item()  # Extract the string value
    categorized_series[category].append(recovered_series[i])

# BUILD THE DATASETS

In [9]:
num_cat_F=len(categorized_series['F'])
num_cat_F

277

-  given -data- -number- -random_state-
- returns -number- random elements from -data-, using -random_state-

In [36]:
def build_random_data(data , number , random_state):
    
    
    indexes=np.arange(len(data))
    
    np.random.seed(random_state)
    select_indexes=np.random.choice(indexes , number , False )
    
    to_return=[]
    
    for i in select_indexes:
        
        to_return.append(data[i])
    
    return to_return

-  given -data- ;  -number- 

- returns a list of 7 elements, each element is a different subset of -number- elements from -data- 

In [40]:
def build_different_sets(data , number):
    to_return=[]
    
    for random_state in [2,309209 , 944, 29090 , 775 , 4733, 66]:  # random_seeds
        d=build_random_data(data , number,random_state)
        
        to_return.append(d)
        
    return to_return
    
     

- apply "build_different_sets" to every category

In [43]:
A=build_different_sets(categorized_series['A'] , 277)
B=build_different_sets(categorized_series['B'] , 277)
C=build_different_sets(categorized_series['C'] , 277)
D=build_different_sets(categorized_series['D'] , 277)
E=build_different_sets(categorized_series['E'] , 277)
F=build_different_sets(categorized_series['F'] , 277)



# Build the model

In [8]:
from tensorflow import keras

-  Function to build the ensemble

In [9]:
class WeightedAverageLayer(tf.keras.layers.Layer):
    def __init__(self, w1, w2, w3, w4, **kwargs):
        super(WeightedAverageLayer, self).__init__(**kwargs)
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3
        self.w4 = w4

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1] + self.w3 * inputs[2] +self.w4 * inputs[3]

-  Import the models

In [10]:
model_best1 = keras.models.load_model('/kaggle/input/models/final_ResnetBidi (2)/ResnetBidi/SubmissionModel')
model_transformer = keras.models.load_model('/kaggle/input/models/transformer/transformer/SubmissionModel')
model_best2 = keras.models.load_model('/kaggle/input/models/ResnetAttention_32filts_8h16dim/ResnetAttention_32filts_8h16dim/SubmissionModel')
model_informer = keras.models.load_model('/kaggle/input/models/Informer-style/Informer-style')
models = [model_best1, model_best2, model_informer , model_transformer]

In [54]:
sequence_length = 200

def create_sequences(data):
    input_sequences = []
    output_sequences = []
    for series in data:
        for i in range(len(series) - sequence_length - 9):  # Considering 9 samples as the prediction horizon
            input_sequences.append(series[i:i + sequence_length])
            output_sequences.append(series[i + sequence_length:i + sequence_length + 9])
    return np.array(input_sequences), np.array(output_sequences)




- Given a list of lists (list of datasets) return a list of dictionaries.
- Each dictionary has two keys :  "input" and "output" -> "input": input sequences (length=200) ; "output": output sequences ( length=9)

In [56]:
def build_input_output(list_of_lists):
    to_return=[]
    
    for el in range(len(list_of_lists)):
        d={}
        
        inp , out = create_sequences(list_of_lists[el])
        
        d["input"]=inp
        
        d["output"]=out
        
        to_return.append(d)
        
    return to_return

In [60]:
A_seq=build_input_output(A)
B_seq=build_input_output(B)
C_seq=build_input_output(C)
D_seq=build_input_output(D)
E_seq=build_input_output(E)
F_seq=build_input_output(F)


In [12]:
inputs = keras.Input(shape=(200,1))
outputs = [model(inputs) for model in models]

-  Function to give a more robust estimation of MSE and MAE of a model

- For each category we have 7 different datasets and we have 6 categories.

- For each category the model is tested on the 7 datasets and the avarage mse and mae are computed

- the final mse and mae are the avarage of the mse and mae obtained for the categories

In [75]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_model(model , list_of_cat_sequences=[A_seq , B_seq , C_seq , D_seq , E_seq , F_seq]):
    MSE=[]
    MAE=[]
    
    for seq in list_of_cat_sequences:
        mse=[]
        mae=[]
        
        for el in seq:
            pred=model.predict(el["input"] , verbose=0)
            mse.append(mean_squared_error(el["output"], pred))
            mae.append(mean_absolute_error(el["output"], pred))
            
        print("done_1")
        MSE.append(np.mean(mse))
        MAE.append(np.mean(mae))
        
    return {"MSE" : np.mean(MSE) , "MAE" : np.mean(MAE)}
        

- different combibation of weights are tested

In [76]:
#0.25,0.25,0.25,0.25
ensemble_output = WeightedAverageLayer(0.25, 0.25, 0.25, 0.25)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')


In [77]:
evaluate_model(model_ensemble)

done_1
done_1
done_1
done_1
done_1
done_1


{'MSE': 0.004184409471205163, 'MAE': 0.03857088708292402}

In [78]:
#(0.4 , 0.2 , 0.2 , 0.2)
ensemble_output = WeightedAverageLayer(0.4 , 0.2 , 0.2 , 0.2)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
evaluate_model(model_ensemble)

done_1
done_1
done_1
done_1
done_1
done_1


{'MSE': 0.004181274182498473, 'MAE': 0.038568294852347265}

In [79]:
#(0.3,0.2,0.25,0.25)
ensemble_output = WeightedAverageLayer(0.3,0.2,0.25,0.25)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
evaluate_model(model_ensemble)

done_1
done_1
done_1
done_1
done_1
done_1


{'MSE': 0.004202270443320931, 'MAE': 0.038656827827364335}

In [80]:
#(0.35 , 0.2 , 0.2, 0.25)
ensemble_output = WeightedAverageLayer(0.35 , 0.2 , 0.2, 0.25)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
evaluate_model(model_ensemble)

done_1
done_1
done_1
done_1
done_1
done_1


{'MSE': 0.00419015493168083, 'MAE': 0.03862259015288376}

In [81]:
#(0.5 , 0.15 , 0.15 , 0.2)
ensemble_output = WeightedAverageLayer(0.5 , 0.15 , 0.15 , 0.2)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
evaluate_model(model_ensemble)

done_1
done_1
done_1
done_1
done_1
done_1


{'MSE': 0.004210482490092536, 'MAE': 0.03876622074312779}

In [82]:
#(0.5 , 0.15 , 0.2 , 0.15)
ensemble_output = WeightedAverageLayer(0.5 , 0.15 , 0.2 , 0.15)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
evaluate_model(model_ensemble)

done_1
done_1
done_1
done_1
done_1
done_1


{'MSE': 0.004212513862727453, 'MAE': 0.038740852678702094}

In [83]:
#(0.4 , 0.175 , 0.25 , 0.175)
ensemble_output = WeightedAverageLayer(0.4 , 0.175 , 0.25 , 0.175)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
evaluate_model(model_ensemble)

done_1
done_1
done_1
done_1
done_1
done_1


{'MSE': 0.004200037390236453, 'MAE': 0.03863299404319757}

In [84]:
#(0.4 , 0.25 , 0.175 , 0.175)
ensemble_output = WeightedAverageLayer(0.4 , 0.25 , 0.175 , 0.175)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
evaluate_model(model_ensemble)

done_1
done_1
done_1
done_1
done_1
done_1


{'MSE': 0.004150239701113451, 'MAE': 0.03842443998627953}

In [85]:
#(0.4 , 0.175 , 0.175 , 0.25)
ensemble_output = WeightedAverageLayer(0.4 , 0.175 , 0.175 , 0.25)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
evaluate_model(model_ensemble)

done_1
done_1
done_1
done_1
done_1
done_1


{'MSE': 0.0041992394558032315, 'MAE': 0.03868756361026331}

In [86]:
#(0.4 , 0.25 , 0.2 , 0.15)
ensemble_output = WeightedAverageLayer(0.4 , 0.25 , 0.2 , 0.15)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
evaluate_model(model_ensemble)

done_1
done_1
done_1
done_1
done_1
done_1


{'MSE': 0.00415122399706361, 'MAE': 0.03841189391653925}

# Best

In [13]:
ensemble_output = WeightedAverageLayer(0.4 , 0.25 , 0.175 , 0.175)(outputs)
model_ensemble = keras.Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
model_ensemble.save('ensemble')